In [1]:
# 라이브러리 설치
!pip install bertopic sentence-transformers umap-learn hdbscan plotly -q

print("✅ Packages installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 3.1 MB/s eta 0:00:00
✅ Packages installed!


In [2]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import plotly.express as px
import plotly.graph_objects as go
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [3]:
from google.colab import files
import pandas as pd

# 파일 업로드
uploaded = files.upload()

Saving Korea_ESG_Natural_3000.csv to Korea_ESG_Natural_3000.csv


In [4]:
from google.colab import files
import pandas as pd

# 파일 업로드
uploaded = files.upload()

Saving Korea_ESG_Natural_3000.csv to Korea_ESG_Natural_3000 (1).csv


In [5]:
# 파일 로드
kr_natural = pd.read_csv('Korea_ESG_Natural_3000.csv')
#uk_natural = pd.read_csv('uk_natural_clean.csv')

# 날짜 형식 변환
kr_natural['date'] = pd.to_datetime(kr_natural['date'])
#uk_natural['date'] = pd.to_datetime(uk_natural['date'])

# year_month를 문자열로 변환했다면 다시 Period로
if kr_natural['year_month'].dtype == 'object':
    kr_natural['year_month'] = pd.to_datetime(kr_natural['year_month']).dt.to_period('M')
    #uk_natural['year_month'] = pd.to_datetime(uk_natural['year_month']).dt.to_period('M')

# 데이터 확인
print("\n" + "="*70)
print("📊 DATA SUMMARY")
print("="*70)

print(f"\n🇰🇷 Korea Natural: {len(kr_natural)} articles")
print(f"   E: {(kr_natural['dominant_category']=='E').sum()} ({(kr_natural['dominant_category']=='E').sum()/len(kr_natural)*100:.1f}%)")
print(f"   S: {(kr_natural['dominant_category']=='S').sum()} ({(kr_natural['dominant_category']=='S').sum()/len(kr_natural)*100:.1f}%)")
print(f"   G: {(kr_natural['dominant_category']=='G').sum()} ({(kr_natural['dominant_category']=='G').sum()/len(kr_natural)*100:.1f}%)")

#print(f"\n🇬🇧 UK Natural: {len(uk_natural)} articles")
#print(f"   E: {(uk_natural['category']=='E').sum()} ({(uk_natural['category']=='E').sum()/len(uk_natural)*100:.1f}%)")
#print(f"   S: {(uk_natural['category']=='S').sum()} ({(uk_natural['category']=='S').sum()/len(uk_natural)*100:.1f}%)")
#print(f"   G: {(uk_natural['category']=='G').sum()} ({(uk_natural['category']=='G').sum()/len(uk_natural)*100:.1f}%)")

print("\n✅ Data ready for BERTopic!")


📊 DATA SUMMARY

🇰🇷 Korea Natural: 3000 articles
   E: 1360 (45.3%)
   S: 556 (18.5%)
   G: 1084 (36.1%)

✅ Data ready for BERTopic!


In [6]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk

In [7]:
# NLTK 불용어 다운로드 (처음 한 번만)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

In [15]:
def get_extended_stopwords():
    """
    영어 불용어를 대폭 확장
    - NLTK 기본 불용어
    - 뉴스 특화 불용어 (said, according, news 등)
    - 짧은 단어들
    """
    # 기본 영어 불용어
    english_stopwords = set(stopwords.words('english'))

    # 뉴스 특화 불용어 추가
    news_stopwords = {
        'said', 'says', 'according', 'told', 'report', 'reports',
        'news', 'article', 'today', 'yesterday', 'new', 'year',
        'also', 'will', 'would', 'could', 'should', 'may', 'might',
        'one', 'two', 'three', 'first', 'second', 'last',
        'mr', 'mrs', 'ms', 'dr', 'says', 'told',
        'uk', 'us', 'britain', 'british', 'american',
        'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
        'january', 'february', 'march', 'april', 'may', 'june',
        'july', 'august', 'september', 'october', 'november', 'december'
    }

    # 1-2글자 단어들 (대부분 불용어)
    short_words = {
        'an', 'as', 'at', 'be', 'by', 'do', 'go', 'he', 'if', 'in',
        'is', 'it', 'me', 'my', 'no', 'of', 'on', 'or', 'so', 'to',
        'up', 'us', 'we'
    }

    # 모두 합치기
    all_stopwords = english_stopwords | news_stopwords | short_words

    return list(all_stopwords)

In [16]:
def create_custom_vectorizer():
    """
    BERTopic용 커스텀 CountVectorizer

    개선사항:
    1. 확장된 불용어 사용
    2. 최소 4글자 이상 단어만 (min_chars=4)
    3. bigram 포함 (1-2 단어 조합)
    4. 최소 5번 이상 등장하는 단어만
    """
    extended_stopwords = get_extended_stopwords()

    vectorizer = CountVectorizer(
        ngram_range=(1, 2),           # unigram + bigram
        stop_words=extended_stopwords, # 확장된 불용어
        min_df=5,                     # 최소 5번 등장
        max_df=0.7,                   # 70% 이상 문서에 등장하면 제외
        token_pattern=r'\b[a-z]{4,}\b' # 4글자 이상만 (짧은 불용어 제거!)
    )

    return vectorizer

In [17]:
def analyze_topics_improved(texts, category, country, n_topics=None):
    """
    개선된 BERTopic 분석

    Parameters:
    -----------
    texts : list of str
        분석할 텍스트들
    category : str
        카테고리 (E/S/G)
    country : str
        국가 (Korea/UK)
    n_topics : int or None
        토픽 개수 (None이면 자동)

    Returns:
    --------
    topic_model, topics, probs
    """
    print(f"\n{'='*60}")
    print(f"🔍 {country} - {category} (Improved)")
    print(f"{'='*60}")
    print(f"Documents: {len(texts)}")

    # 1. Embedding model
    embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    # 2. 커스텀 vectorizer (영국 데이터용 강화!)
    if country == "UK":
        vectorizer = create_custom_vectorizer()
        print(f"📝 Using custom vectorizer with {len(get_extended_stopwords())} stopwords")
    else:
        # 한국어는 기본 설정으로도 잘 됨
        vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=3)

    # 3. BERTopic 모델
    topic_model = BERTopic(
        embedding_model=embedding_model,
        vectorizer_model=vectorizer,    # 커스텀 vectorizer!
        min_topic_size=20,              # 약간 줄임 (더 세분화)
        nr_topics=n_topics,
        calculate_probabilities=True,
        verbose=False
    )

    # 4. 실행
    print(f"🔄 Running BERTopic with {'content' if country == 'UK' else 'analysis_text'}... (2-4 minutes)")
    topics, probs = topic_model.fit_transform(texts)

    # 5. 결과
    topic_info = topic_model.get_topic_info()
    n_actual_topics = len(topic_info[topic_info['Topic'] != -1])
    topics_array = np.array(topics)
    outliers = (topics_array == -1).sum()

    print(f"\n✅ Found {n_actual_topics} topics")
    print(f"   Outliers: {outliers} ({outliers/len(texts)*100:.1f}%)")

    # Top 8 토픽 출력
    print(f"\n📊 Top 8 Topics:")
    for idx, row in topic_info[topic_info['Topic'] != -1].head(8).iterrows():
        print(f"  Topic {row['Topic']:2d}: {row['Count']:4d} docs - {row['Name']}")

    return topic_model, topics, probs

In [18]:
def main():
    """
    개선된 분석 실행

    전제: kr_e, uk_e DataFrame이 이미 로드되어 있어야 함
    """
    print("\n" + "="*70)
    print("🇰🇷 KOREA E - Content Analysis")
    print("="*70)

    # 한국: analysis_text 사용
    kr_texts = kr_e['analysis_text'].fillna('').astype(str).tolist()

    print(f"\n{'='*60}")
    print(f"🔍 Korea - E (Improved)")
    print(f"{'='*60}")
    print(f"Documents: {len(kr_texts)}")
    print("🔄 Running BERTopic with analysis_text... (2-4 minutes)")

    # 한국 분석
    kr_model, kr_topics, kr_probs = analyze_topics_improved(
        kr_texts,
        category='E',
        country='Korea',
        n_topics=None  # 자동
    )

    print("\n" + "="*70)
    print("🇬🇧 UK E - Content Analysis")
    print("="*70)

    # 영국: content 사용 (더 김!)
    #uk_texts = uk_e['content'].fillna('').astype(str).tolist()

    #print(f"\n🇬🇧 Using content for UK")
    #print(f"\n📊 Text length check:")
    #print(f"   Korea avg length: {np.mean([len(t) for t in kr_texts]):.0f} chars")
    #print(f"   UK avg length: {np.mean([len(t) for t in uk_texts]):.0f} chars")

    # 영국 분석 (개선된 vectorizer!)
    #uk_model, uk_topics, uk_probs = analyze_topics_improved(
        #uk_texts,
        #category='E',
        #country='UK',
        #n_topics=None  # 자동
    #)

    print("\n✅ Improved analysis complete!")

    return kr_model, kr_topics #uk_model, uk_topics

In [ ]:
# 2. 개선된 분석 실행
kr_model, kr_topics = main()

# 3. 결과 확인
print("\n🎉 Done! Check the topics above.")
print("\n💡 Tips:")
print("- Korea topics should show: ESG, 투자, 탄소, 기후 등")
print("- UK topics should show: climate, energy, carbon, emissions 등")
print("- NOT 'the, to, and, of' anymore!")


🇰🇷 KOREA E - Content Analysis


NameError: name 'kr_e' is not defined

In [ ]:
# 1. uk_e 샘플 확인
print("🇬🇧 UK E 카테고리 샘플 확인:")
print(f"Total: {len(uk_e)} articles")
print("\n📰 Sample titles:")
for i, row in uk_e.sample(10).iterrows():
    print(f"\n{i}. {row['title']}")
    print(f"   Keywords: {row.get('matched_keywords', 'N/A')}")

# 2. E 키워드 매칭 빈도 확인
if 'matched_keywords' in uk_e.columns:
    print("\n📊 Most common E keywords:")
    all_keywords = []
    for keywords in uk_e['matched_keywords'].dropna():
        if isinstance(keywords, str):
            all_keywords.extend(keywords.split(','))

    from collections import Counter
    keyword_counts = Counter(all_keywords)
    for keyword, count in keyword_counts.most_common(10):
        print(f"  {keyword.strip()}: {count}")

🇬🇧 UK E 카테고리 샘플 확인:
Total: 2240 articles

📰 Sample titles:

1782. World Bank staff were told to give special treatment to son of Trump official
   Keywords: N/A

2437.  Kerry Kennedy on the family political split: ‘There’s so much at stake’
   Keywords: N/A

677. Joe Biden nominates former Mastercard boss Ajay Banga to lead World Bank
   Keywords: N/A

1466. How do you feel about emotional affairs? Tell us about your experiences
   Keywords: N/A

2528. Shell accused of ‘profiteering bonanza’ after record first-quarter profits of $9.6bn
   Keywords: N/A

1774. More than 800m Amazon trees felled in six years to meet beef demand
   Keywords: N/A

439. Dutch airline KLM misled customers with vague green claims, court rules
   Keywords: N/A

2223. The Taylor Swift gig economy is so big it’s even causing geopolitical tensions | Zing Tsjeng
   Keywords: N/A

802. Burned, suffocated, beaten: why women in Lebanon are dying at the hands of their partners
   Keywords: N/A

416. Tomás Saraceno: We

In [ ]:
# 1. uk_e가 어떻게 만들어졌는지 확인
print("Original filtering code:")
print("uk_e = ???")  # 어떤 코드로 만들었나요?

# 2. 컬럼 확인
print(f"\nColumns: {uk_e.columns.tolist()}")

# 3. 원본 데이터 확인
if 'uk_natural' in globals():
    print(f"\nuk_natural exists: {len(uk_natural)} articles")
    print(f"Categories: {uk_natural['category'].value_counts()}")

Original filtering code:
uk_e = ???

Columns: ['id', 'type', 'outlet', 'date', 'title', 'url', 'headline', 'content', 'trail_text', 'wordcount', 'date_uk', 'year_month', 'year', 'month', 'content_length', 'analysis_text', 'analysis_text_length', 'e_score', 's_score', 'g_score', 'total_score', 'category', 'country', 'quarter']

uk_natural exists: 2776 articles
Categories: category
E    2240
S     362
G     174
Name: count, dtype: int64


In [ ]:
# 1. 의심스러운 기사들의 스코어 확인
print("🔍 비환경처럼 보이는 기사들의 스코어:")
suspicious_indices = [1782, 2437, 677, 1466, 2223, 802]

for idx in suspicious_indices:
    if idx in uk_e.index:
        row = uk_e.loc[idx]
        print(f"\n📰 {row['title'][:80]}")
        print(f"   E: {row['e_score']:.2f} | S: {row['s_score']:.2f} | G: {row['g_score']:.2f}")
        print(f"   Category: {row['category']}")

# 2. E 카테고리 전체 스코어 분포
print("\n\n📊 UK E 카테고리 스코어 분포:")
print(uk_e[['e_score', 's_score', 'g_score']].describe())

# 3. e_score가 낮은 기사들 (잘못 분류됐을 가능성)
print("\n\n🚨 e_score가 낮은데 E로 분류된 기사들:")
low_e = uk_e[uk_e['e_score'] < 1.0].sort_values('e_score')
print(f"Total: {len(low_e)} articles")
for idx, row in low_e.head(10).iterrows():
    print(f"\n  E={row['e_score']:.2f}, S={row['s_score']:.2f}, G={row['g_score']:.2f}")
    print(f"  {row['title'][:100]}")

🔍 비환경처럼 보이는 기사들의 스코어:

📰 World Bank staff were told to give special treatment to son of Trump official
   E: 5.00 | S: 2.00 | G: 1.00
   Category: E

📰  Kerry Kennedy on the family political split: ‘There’s so much at stake’
   E: 5.00 | S: 2.00 | G: 0.00
   Category: E

📰 Joe Biden nominates former Mastercard boss Ajay Banga to lead World Bank
   E: 11.00 | S: 0.00 | G: 2.00
   Category: E

📰 How do you feel about emotional affairs? Tell us about your experiences
   E: 5.00 | S: 0.00 | G: 1.00
   Category: E

📰 The Taylor Swift gig economy is so big it’s even causing geopolitical tensions |
   E: 23.00 | S: 9.00 | G: 2.00
   Category: E

📰 Burned, suffocated, beaten: why women in Lebanon are dying at the hands of their
   E: 47.00 | S: 1.00 | G: 1.00
   Category: E


📊 UK E 카테고리 스코어 분포:
           e_score      s_score      g_score
count  2240.000000  2240.000000  2240.000000
mean     13.575000     3.037946     2.158482
std      11.746949     3.914436     2.890030
min       1.000000   

In [12]:
def analyze_topics_bertopic(texts, category, country, n_topics='auto'):
    """BERTopic으로 주제 분석"""

    print(f"\n{'='*60}")
    print(f"🔍 {country} - {category} Category Analysis")
    print(f"{'='*60}")
    print(f"Documents: {len(texts)}")

    # Multilingual embedding model
    embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    # BERTopic 모델 (파라미터 조정!)
    if n_topics == 'auto':
        topic_model = BERTopic(
            embedding_model=embedding_model,
            min_topic_size=30,  # 최소 문서 수 늘림 (15 → 30)
            nr_topics=None,     # 자동 결정
            calculate_probabilities=True,
            verbose=False
        )
    else:
        topic_model = BERTopic(
            embedding_model=embedding_model,
            min_topic_size=30,
            nr_topics=n_topics,
            calculate_probabilities=True,
            verbose=False
        )

    # 실행
    print("🔄 Running BERTopic... (1-3 minutes)")
    topics, probs = topic_model.fit_transform(texts)

    # 결과
    topic_info = topic_model.get_topic_info()
    n_actual_topics = len(topic_info[topic_info['Topic'] != -1])

    # numpy array로 변환해서 sum() 사용
    topics_array = np.array(topics)
    n_outliers = (topics_array == -1).sum()

    print(f"\n✅ Found {n_actual_topics} topics")
    print(f"   Outliers: {n_outliers} documents ({n_outliers/len(texts)*100:.1f}%)")

    print(f"\n📊 Top 5 Topics:")
    for idx, row in topic_info[topic_info['Topic'] != -1].head(5).iterrows():
        print(f"  Topic {row['Topic']:2d}: {row['Count']:4d} docs - {row['Name'][:60]}")

    return topic_model, topics, probs

print("✅ Function updated and fixed!")

✅ Function updated and fixed!


In [19]:
print("\n" + "="*70)
print("🌱 ENVIRONMENT (E) CATEGORY - BERTOPIC ANALYSIS")
print("="*70)

# 데이터 필터링
kr_e = kr_natural[kr_natural['dominant_category'] == 'E'].copy()
#uk_e = uk_natural[uk_natural['category'] == 'E'].copy()

print(f"\n📊 Sample sizes:")
print(f"   Korea E: {len(kr_e)} articles")
#print(f"   UK E: {len(uk_e)} articles")

# 텍스트 추출 (title 사용 - 빠르고 효과적!)
# content 전체 쓰면 더 정확하지만 시간 오래 걸림
kr_e_texts = kr_e['title'].fillna('').astype(str).tolist()
#uk_e_texts = uk_e['title'].fillna('').astype(str).tolist()

print("\n" + "="*70)
print("🇰🇷 KOREA - Environment Topics")
print("="*70)

# 한국 E 분석
kr_e_model, kr_e_topics, kr_e_probs = analyze_topics_bertopic(
    kr_e_texts,
    category='E',
    country='Korea',
    n_topics=10
)

#print("\n" + "="*70)
#print("🇬🇧 UK - Environment Topics")
#print("="*70)

# 영국 E 분석
#uk_e_model, uk_e_topics, uk_e_probs = analyze_topics_bertopic(
    #uk_e_texts,
    #category='E',
    #country='UK',
    #n_topics=10
#)

print("\n✅ Environment category analysis complete!")


🌱 ENVIRONMENT (E) CATEGORY - BERTOPIC ANALYSIS

📊 Sample sizes:
   Korea E: 1360 articles

🇰🇷 KOREA - Environment Topics

🔍 Korea - E Category Analysis
Documents: 1360
🔄 Running BERTopic... (1-3 minutes)

✅ Found 2 topics
   Outliers: 12 documents (0.9%)

📊 Top 5 Topics:
  Topic  0:  214 docs - 0_esg_경영_강화_평가
  Topic  1: 1134 docs - 1_친환경_투자_지원_기업

✅ Environment category analysis complete!


In [20]:
# ============================================================
# 한국 E - 지난번 설정으로 재분석
# ============================================================

print("="*70)
print("🇰🇷 한국 E - 지난번 방식으로 재분석")
print("="*70)

# 데이터 확인
print(f"\n📊 데이터 개수 확인:")
print(f"   kr_e: {len(kr_e):,}개")

# 텍스트 추출
if 'analysis_text' in kr_e.columns:
    kr_texts = kr_e['analysis_text'].fillna('').astype(str).tolist()
    print("   Using: analysis_text")
else:
    kr_texts = kr_e['content'].fillna('').astype(str).tolist()
    print("   Using: content")

print(f"   최종: {len(kr_texts):,}개")

# BERTopic 설정 (지난번과 동일하게)
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 핵심: min_topic_size를 더 작게!
topic_model_kr = BERTopic(
    embedding_model=embedding_model,
    min_topic_size=15,  # 20 → 15 (또는 10)
    nr_topics='auto',
    calculate_probabilities=True,
    verbose=True
)

print(f"\n🔄 BERTopic 실행 중...")
kr_topics, kr_probs = topic_model_kr.fit_transform(kr_texts)

# 결과 확인
kr_topic_info = topic_model_kr.get_topic_info()
kr_valid = kr_topic_info[kr_topic_info['Topic'] != -1]
kr_outliers = np.sum(kr_topics == -1)

print(f"\n✅ 결과:")
print(f"   토픽: {len(kr_valid)}개")
print(f"   Outliers: {kr_outliers}개 ({kr_outliers/len(kr_texts)*100:.1f}%)")

print(f"\n📊 상위 12개 토픽:")
for idx, row in kr_valid.head(12).iterrows():
    print(f"  Topic {row['Topic']:2d}: {row['Count']:4d} docs - {row['Name'][:70]}")

# 비교
print(f"\n" + "="*70)
print("📊 결과 비교")
print("="*70)
print(f"           이번    지난번   목표")
print(f"토픽:      {len(kr_valid):2d}개    12개     12개")
print(f"Outliers:  {kr_outliers/len(kr_texts)*100:5.1f}%   42.1%    ~40%")

🇰🇷 한국 E - 지난번 방식으로 재분석

📊 데이터 개수 확인:
   kr_e: 1,360개
   Using: analysis_text
   최종: 1,360개


2025-11-06 15:25:39,707 - BERTopic - Embedding - Transforming documents to embeddings.



🔄 BERTopic 실행 중...


Batches:   0%|          | 0/43 [00:00<?, ?it/s]

2025-11-06 15:28:20,732 - BERTopic - Embedding - Completed ✓
2025-11-06 15:28:20,736 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-06 15:28:25,412 - BERTopic - Dimensionality - Completed ✓
2025-11-06 15:28:25,414 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-06 15:28:25,592 - BERTopic - Cluster - Completed ✓
2025-11-06 15:28:25,593 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-11-06 15:28:26,405 - BERTopic - Representation - Completed ✓
2025-11-06 15:28:26,408 - BERTopic - Topic reduction - Reducing number of topics
2025-11-06 15:28:26,426 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-06 15:28:27,185 - BERTopic - Representation - Completed ✓
2025-11-06 15:28:27,189 - BERTopic - Topic reduction - Reduced number of topics from 14 to 14



✅ 결과:
   토픽: 13개
   Outliers: 0개 (0.0%)

📊 상위 12개 토픽:
  Topic  0:  144 docs - 0_펀드_투자_esg_미국
  Topic  1:  113 docs - 1_esg_기업_환경_경영
  Topic  2:  106 docs - 2_탄소_기후_에너지_기업
  Topic  3:   88 docs - 3_mou_협력_체결_협약
  Topic  4:   79 docs - 4_플라스틱_제품_사용_소재
  Topic  5:   60 docs - 5_기업_투자_시장_스타트업
  Topic  6:   57 docs - 6_금융_회장_기업_지원
  Topic  7:   46 docs - 7_평가_등급_esg_획득
  Topic  8:   32 docs - 8_배터리_lg전자_lg_전기차
  Topic  9:   30 docs - 9_교육_프로그램_학생들_학생
  Topic 10:   25 docs - 10_sk_skc_사업_sk에코플랜트
  Topic 11:   25 docs - 11_태양광_에너지_re100_재생에너지

📊 결과 비교
           이번    지난번   목표
토픽:      13개    12개     12개
Outliers:    0.0%   42.1%    ~40%


In [21]:
# 텍스트 추출: content 또는 analysis_text 사용

# 한국 E - analysis_text 사용
if 'analysis_text' in kr_e.columns:
    kr_e_texts_improved = kr_e['analysis_text'].fillna('').astype(str).tolist()
    print("🇰🇷 Using analysis_text for Korea")
else:
    kr_e_texts_improved = kr_e['content'].fillna('').astype(str).tolist()
    print("🇰🇷 Using content for Korea")

# 영국 E - content 사용
uk_e_texts_improved = uk_e['content'].fillna('').astype(str).tolist()
print("🇬🇧 Using content for UK")

print(f"\n📊 Text length check:")
print(f"   Korea avg length: {np.mean([len(t) for t in kr_e_texts_improved[:100]]):.0f} chars")
print(f"   UK avg length: {np.mean([len(t) for t in uk_e_texts_improved[:100]]):.0f} chars")

# 파라미터 조정된 함수
def analyze_topics_improved(texts, category, country, n_topics='auto', min_size=20):
    """개선된 BERTopic (더 작은 min_topic_size)"""

    print(f"\n{'='*60}")
    print(f"🔍 {country} - {category} (Improved)")
    print(f"{'='*60}")
    print(f"Documents: {len(texts)}")

    embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    topic_model = BERTopic(
        embedding_model=embedding_model,
        min_topic_size=min_size,  # 20으로 줄임!
        nr_topics=None if n_topics == 'auto' else n_topics,
        calculate_probabilities=True,
        verbose=False
    )

    print("🔄 Running BERTopic with content... (2-4 minutes)")
    topics, probs = topic_model.fit_transform(texts)

    topic_info = topic_model.get_topic_info()
    n_actual_topics = len(topic_info[topic_info['Topic'] != -1])

    topics_array = np.array(topics)
    n_outliers = (topics_array == -1).sum()

    print(f"\n✅ Found {n_actual_topics} topics")
    print(f"   Outliers: {n_outliers} ({n_outliers/len(texts)*100:.1f}%)")

    print(f"\n📊 Top 8 Topics:")
    for idx, row in topic_info[topic_info['Topic'] != -1].head(8).iterrows():
        print(f"  Topic {row['Topic']:2d}: {row['Count']:4d} docs - {row['Name'][:70]}")

    return topic_model, topics, probs

# 한국 E 재분석
print("\n" + "="*70)
print("🇰🇷 KOREA E - Content Analysis")
print("="*70)

kr_e_model_v2, kr_e_topics_v2, kr_e_probs_v2 = analyze_topics_improved(
    kr_e_texts_improved,
    category='E',
    country='Korea',
    n_topics='auto',
    min_size=20  # 30 → 20
)

# 영국 E 재분석
print("\n" + "="*70)
print("🇬🇧 UK E - Content Analysis")
print("="*70)

uk_e_model_v2, uk_e_topics_v2, uk_e_probs_v2 = analyze_topics_improved(
    uk_e_texts_improved,
    category='E',
    country='UK',
    n_topics='auto',
    min_size=20
)

print("\n✅ Improved analysis complete!")

🇰🇷 Using analysis_text for Korea


NameError: name 'uk_e' is not defined

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

print("\n" + "="*70)
print("🔧 UK E - IMPROVED with Custom Vectorizer")
print("="*70)

# 추가 불용어 정의
custom_stop_words = [
    'said', 'says', 'told', 'say', 'saying',
    'year', 'years', 'day', 'days', 'week', 'weeks', 'month', 'months',
    'new', 'first', 'last', 'second', 'third', 'two', 'three', 'one',
    'people', 'man', 'woman', 'men', 'women',
    'uk', 'us', 'britain', 'british', 'america', 'american',
    'government', 'minister', 'prime', 'party', 'labour', 'conservative',
    'make', 'makes', 'made', 'making', 'take', 'takes', 'took',
    'get', 'gets', 'got', 'give', 'gives', 'gave',
    'could', 'would', 'should', 'may', 'might', 'must',
    'way', 'ways', 'thing', 'things', 'good', 'bad', 'best', 'worst',
    'guardian', 'report', 'reports', 'review', 'reviews'
]

# sklearn 기본 불용어 + 커스텀 불용어 합치기
all_stop_words = list(ENGLISH_STOP_WORDS.union(set(custom_stop_words)))

print(f"📝 Total stopwords: {len(all_stop_words)}")

# Custom vectorizer (수정!)
vectorizer_model = CountVectorizer(
    stop_words=all_stop_words,  # 합친 리스트 직접 넘기기!
    min_df=5,
    max_df=0.5,
    ngram_range=(1, 2),
    token_pattern=r'\b[a-zA-Z]{4,}\b'  # 4글자 이상만
)

# Embedding model
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# BERTopic with custom vectorizer
topic_model_uk_final = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    min_topic_size=25,
    nr_topics=None,
    calculate_probabilities=True,
    verbose=False
)

print("\n🔄 Running improved BERTopic for UK... (3-5 minutes)")
print("   (Strong stopword removal + 4+ letter words only)")

topics_uk_final, probs_uk_final = topic_model_uk_final.fit_transform(uk_e_texts_improved)

# 결과
topic_info_uk = topic_model_uk_final.get_topic_info()
n_topics_uk = len(topic_info_uk[topic_info_uk['Topic'] != -1])
n_outliers_uk = (np.array(topics_uk_final) == -1).sum()

print(f"\n✅ Found {n_topics_uk} topics")
print(f"   Outliers: {n_outliers_uk} ({n_outliers_uk/len(uk_e_texts_improved)*100:.1f}%)")

print(f"\n📊 Top 10 Topics:")
for idx, row in topic_info_uk[topic_info_uk['Topic'] != -1].head(10).iterrows():
    print(f"  Topic {row['Topic']:2d}: {row['Count']:4d} docs - {row['Name'][:75]}")

# 상세 키워드도 보기
print(f"\n📋 Detailed Keywords (Top 5 Topics):")
for idx, row in topic_info_uk[topic_info_uk['Topic'] != -1].head(5).iterrows():
    topic_id = row['Topic']
    topic_words = topic_model_uk_final.get_topic(topic_id)
    if topic_words:
        keywords = [word for word, score in topic_words[:5]]
        print(f"\nTopic {topic_id}: {row['Name'][:50]}")
        print(f"  Keywords: {', '.join(keywords)}")
        print(f"  Count: {row['Count']} documents")

print("\n✅ UK E analysis complete!")


🔧 UK E - IMPROVED with Custom Vectorizer
📝 Total stopwords: 371

🔄 Running improved BERTopic for UK... (3-5 minutes)
   (Strong stopword removal + 4+ letter words only)

✅ Found 10 topics
   Outliers: 706 (31.5%)

📊 Top 10 Topics:
  Topic  0:  630 docs - 0_reeves_diesel_battery_northern ireland
  Topic  1:  244 docs - 1_novel_characters_ballet_netflix
  Topic  2:  236 docs - 2_medal_champions league_championship_finals
  Topic  3:  132 docs - 3_national park_reefs_rewilding_roam
  Topic  4:   72 docs - 4_greene_chicago_nominee_presidency
  Topic  5:   50 docs - 5_netanyahu_palestinians_hostages_benjamin netanyahu
  Topic  6:   50 docs - 6_paramount_sanders_automated_tweets
  Topic  7:   49 docs - 7_kremlin_troops_frontline_nuclear weapons
  Topic  8:   41 docs - 8_anglo_discharge_leisure_runoff
  Topic  9:   30 docs - 9_chilli_sauce_salad_tomatoes

📋 Detailed Keywords (Top 5 Topics):

Topic 0: 0_reeves_diesel_battery_northern ireland
  Keywords: reeves, diesel, battery, northern irela

In [22]:
# ============================================================
# 한국 S & G - 데이터 현황 확인
# ============================================================

print("="*70)
print("🇰🇷 한국 S & G 카테고리 현황")
print("="*70)

# 전체 데이터에서 S, G 추출
print(f"\n📊 전체 데이터:")
print(f"   총: {len(kr_natural):,}개")

# S 카테고리
kr_s_all = kr_natural[kr_natural['dominant_category'] == 'S'].copy()
print(f"\n📌 S 카테고리: {len(kr_s_all):,}개")

# G 카테고리
kr_g_all = kr_natural[kr_natural['dominant_category'] == 'G'].copy()
print(f"📌 G 카테고리: {len(kr_g_all):,}개")

# ============================================================
# S 카테고리 샘플 확인 (영국처럼 문제 있는지?)
# ============================================================

print(f"\n" + "="*70)
print("🔍 한국 S - 랜덤 샘플 20개")
print("="*70)

if len(kr_s_all) > 0:
    kr_s_clean = kr_s_all.dropna(subset=['title'])

    sample_s = kr_s_clean.sample(min(20, len(kr_s_clean)), random_state=60)

    for idx, row in sample_s.iterrows():
        print(f"\n{row['title']}")
        if 's_score' in row:
            print(f"  S점수: {row.get('s_score', 'N/A')}")

    # 의심 키워드 체크
    print(f"\n⚠️ 오분류 가능성 체크:")

    # 스포츠
    sports = kr_s_clean['title'].str.contains('축구|야구|농구|올림픽|스포츠', na=False).sum()
    print(f"  스포츠: {sports}개")

    # 엔터테인먼트
    culture = kr_s_clean['title'].str.contains('영화|드라마|소설|음악|공연', na=False).sum()
    print(f"  엔터: {culture}개")

    # 환경 (E여야 함)
    env = kr_s_clean['title'].str.contains('기후|탄소|환경오염|재생에너지', na=False).sum()
    print(f"  환경(E): {env}개")

else:
    print("  ❌ S 카테고리 데이터 없음!")

# ============================================================
# G 카테고리 샘플 확인
# ============================================================

print(f"\n" + "="*70)
print("🔍 한국 G - 랜덤 샘플 20개")
print("="*70)

if len(kr_g_all) > 0:
    kr_g_clean = kr_g_all.dropna(subset=['title'])

    sample_g = kr_g_clean.sample(min(20, len(kr_g_clean)), random_state=61)

    for idx, row in sample_g.iterrows():
        print(f"\n{row['title']}")
        if 'g_score' in row:
            print(f"  G점수: {row.get('g_score', 'N/A')}")

    # 의심 키워드 체크
    print(f"\n⚠️ 오분류 가능성 체크:")

    sports = kr_g_clean['title'].str.contains('축구|야구|농구|올림픽', na=False).sum()
    print(f"  스포츠: {sports}개")

    culture = kr_g_clean['title'].str.contains('영화|드라마|소설|음악', na=False).sum()
    print(f"  엔터: {culture}개")

    env = kr_g_clean['title'].str.contains('기후|탄소|환경', na=False).sum()
    print(f"  환경(E): {env}개")

    social = kr_g_clean['title'].str.contains('노동|다양성|인권', na=False).sum()
    print(f"  사회(S): {social}개")

else:
    print("  ❌ G 카테고리 데이터 없음!")

# ============================================================
# 요약
# ============================================================

print(f"\n" + "="*70)
print("📊 한국 ESG 데이터 요약")
print("="*70)

summary_kr = {
    'Category': ['E', 'S', 'G'],
    'Count': [
        len(kr_e) if 'kr_e' in locals() else 0,
        len(kr_s_all),
        len(kr_g_all)
    ]
}

import pandas as pd
summary_kr_df = pd.DataFrame(summary_kr)
print(summary_kr_df.to_string(index=False))

print(f"\n💡 다음 단계:")
if len(kr_s_all) > 100:
    print(f"  ✅ S 분석 가능 ({len(kr_s_all):,}개)")
else:
    print(f"  ⚠️ S 데이터 부족 ({len(kr_s_all):,}개)")

if len(kr_g_all) > 100:
    print(f"  ✅ G 분석 가능 ({len(kr_g_all):,}개)")
else:
    print(f"  ⚠️ G 데이터 부족 ({len(kr_g_all):,}개)")

🇰🇷 한국 S & G 카테고리 현황

📊 전체 데이터:
   총: 3,000개

📌 S 카테고리: 556개
📌 G 카테고리: 1,084개

🔍 한국 S - 랜덤 샘플 20개

네오위즈, 여성가족부 ‘가족친화인증 기업’ 인증 획득
  S점수: 7

"추첨통해 산모에 평생 숙박권" 저출산에 민간도 '웰컴 베이비'
  S점수: 10

이달곤 동반위원장 “대중소기업 동반성장, 포지티브섬 게임 접근해야” 취임 일성
  S점수: 12

효성, 대중소기업 및 농어촌 상생협력기금 100억 출연 ESG 경영 지속
  S점수: 28

BNK금융그룹, 해외법인 현장점검 및 신성장동력 발굴 지원
  S점수: 6

코웨이, 파트너사 동반성장 ESG 역량 강화 교육
  S점수: 13

코웨이, '동반성장 데이' 열고 협력 방안등 논의
  S점수: 5

‘하나 소셜벤처’ 우수 기업 지역 일자리 창출 기대
  S점수: 6

중소벤처기업인증원, 노사상생인증 자문위원회 발족
  S점수: 23

SM상선(주) 건설부문, 올해 '중대재해 제로' 선포
  S점수: 18

광명시, 민생경제 활성화 위한 2025년 정책 발굴
  S점수: 5

프랭클린 “한국, 내년 2분기 기준금리 인하 예상 채권 투자 매력적”
  S점수: 2

[더벨]'사회적 역할 확대' 에코프로, 시각장애인 교구재 기증
  S점수: 5

국내 AI기업의 해외시장 교두보로 떠오른 중동
  S점수: 1

NH농협은행, 빗썸과 가상자산 이용자예치금 관리계약
  S점수: 2

“중소기업협동조합 활성화 위해 공동사업 지원 확대해야”
  S점수: 2

근로복지공단, 카카오뱅크와 푸른씨앗 도입 확산
  S점수: 13

유한대 '소비자가 선택한 2024년 최고의 브랜드' 선정
  S점수: 9

경기도일자리재단, 부천시 심곡동 서부사업본부 개소
  S점수: 21

프랜차이즈 남다른감자탕, "'형제점과 신뢰 상생 우선 정책'으로 지속 성장"
  S점수: 9

⚠️ 오분류 가능성 체크:
  스포츠: 1개
  엔터: 1개
  환경(E): 1개

🔍 한

In [24]:
# ============================================================
# 한국 S & G - 간단 필터링 + BERTopic
# ============================================================

print("="*70)
print("🇰🇷 한국 S 카테고리 - 필터링 & BERTopic")
print("="*70)

# S 필터링 (간단하게)
kr_s_filtered = kr_s_all.copy()

# 명백한 오분류만 제거
exclude_s = ['축구', '야구', '농구', '올림픽', '영화', '드라마']
for keyword in exclude_s:
    mask = kr_s_filtered['title'].str.contains(keyword, na=False)
    removed = mask.sum()
    if removed > 0:
        kr_s_filtered = kr_s_filtered[~mask]
        print(f"  '{keyword}' 제거: {removed}개")

print(f"\n필터링 결과: {len(kr_s_all)}개 → {len(kr_s_filtered)}개")

# 텍스트 추출
if 'analysis_text' in kr_s_filtered.columns:
    kr_s_texts = kr_s_filtered['analysis_text'].fillna('').astype(str).tolist()
else:
    kr_s_texts = kr_s_filtered['content'].fillna('').astype(str).tolist()

# BERTopic 실행
print(f"\n🔄 BERTopic 실행 중... (1-2분)")

kr_s_model = BERTopic(
    embedding_model=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2'),
    min_topic_size=15,
    nr_topics='auto',
    verbose=True
)

kr_s_topics, kr_s_probs = kr_s_model.fit_transform(kr_s_texts)

# 결과
kr_s_info = kr_s_model.get_topic_info()
kr_s_valid = kr_s_info[kr_s_info['Topic'] != -1]
kr_s_outliers = np.sum(kr_s_topics == -1)

print(f"\n✅ 한국 S 결과:")
print(f"   토픽: {len(kr_s_valid)}개")
print(f"   Outliers: {kr_s_outliers}개 ({kr_s_outliers/len(kr_s_texts)*100:.1f}%)")

print(f"\n📊 상위 토픽들:")
for idx, row in kr_s_valid.head(8).iterrows():
    print(f"  Topic {row['Topic']:2d}: {row['Count']:3d} docs - {row['Name'][:60]}")

# ============================================================
# 한국 G 카테고리 - 필터링 & BERTopic
# ============================================================

print(f"\n" + "="*70)
print("🇰🇷 한국 G 카테고리 - 필터링 & BERTopic")
print("="*70)

# G 필터링 (환경 관련만 제거)
kr_g_filtered = kr_g_all.copy()

# G 점수 낮고 환경 키워드 있는 것만 제거
if 'g_score' in kr_g_filtered.columns:
    env_keywords = ['기후변화', '탄소중립', '재생에너지', '환경오염']

    for keyword in env_keywords:
        mask = (kr_g_filtered['title'].str.contains(keyword, na=False)) & \
               (kr_g_filtered['g_score'] < 15)  # 점수 낮은 것만
        removed = mask.sum()
        if removed > 0:
            kr_g_filtered = kr_g_filtered[~mask]
            print(f"  '{keyword}' (G<15) 제거: {removed}개")

print(f"\n필터링 결과: {len(kr_g_all)}개 → {len(kr_g_filtered)}개")

# 텍스트 추출
if 'analysis_text' in kr_g_filtered.columns:
    kr_g_texts = kr_g_filtered['analysis_text'].fillna('').astype(str).tolist()
else:
    kr_g_texts = kr_g_filtered['content'].fillna('').astype(str).tolist()

# BERTopic 실행
print(f"\n🔄 BERTopic 실행 중... (2-3분)")

kr_g_model = BERTopic(
    embedding_model=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2'),
    min_topic_size=5,  # G가 더 많으니 25
    nr_topics='auto',
    verbose=True
)

kr_g_topics, kr_g_probs = kr_g_model.fit_transform(kr_g_texts)

# 결과
kr_g_info = kr_g_model.get_topic_info()
kr_g_valid = kr_g_info[kr_g_info['Topic'] != -1]
kr_g_outliers = np.sum(kr_g_topics == -1)

print(f"\n✅ 한국 G 결과:")
print(f"   토픽: {len(kr_g_valid)}개")
print(f"   Outliers: {kr_g_outliers}개 ({kr_g_outliers/len(kr_g_texts)*100:.1f}%)")

print(f"\n📊 상위 토픽들:")
for idx, row in kr_g_valid.head(8).iterrows():
    print(f"  Topic {row['Topic']:2d}: {row['Count']:3d} docs - {row['Name'][:60]}")

🇰🇷 한국 S 카테고리 - 필터링 & BERTopic
  '야구' 제거: 1개

필터링 결과: 556개 → 555개

🔄 BERTopic 실행 중... (1-2분)


2025-11-06 15:33:25,443 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

2025-11-06 15:34:30,882 - BERTopic - Embedding - Completed ✓
2025-11-06 15:34:30,884 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-06 15:34:32,580 - BERTopic - Dimensionality - Completed ✓
2025-11-06 15:34:32,581 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-06 15:34:32,618 - BERTopic - Cluster - Completed ✓
2025-11-06 15:34:32,620 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-11-06 15:34:32,981 - BERTopic - Representation - Completed ✓
2025-11-06 15:34:32,983 - BERTopic - Topic reduction - Reducing number of topics
2025-11-06 15:34:32,996 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-06 15:34:33,352 - BERTopic - Representation - Completed ✓
2025-11-06 15:34:33,356 - BERTopic - Topic reduction - Reduced number of topics from 7 to 7



✅ 한국 S 결과:
   토픽: 6개
   Outliers: 0개 (0.0%)

📊 상위 토픽들:
  Topic  0: 147 docs - 0_지원_상생_경영_esg
  Topic  1:  80 docs - 1_esg_협력사_지원_경영
  Topic  2:  51 docs - 2_브랜드_기업_제품_소비자
  Topic  3:  50 docs - 3_투자_기업_지원_채권
  Topic  4:  20 docs - 4_안전_현장_교육_환자
  Topic  5:  16 docs - 5_장애인_전달_사회_기부

🇰🇷 한국 G 카테고리 - 필터링 & BERTopic

필터링 결과: 1084개 → 1084개

🔄 BERTopic 실행 중... (2-3분)


2025-11-06 15:34:35,696 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

2025-11-06 15:36:51,115 - BERTopic - Embedding - Completed ✓
2025-11-06 15:36:51,117 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-06 15:36:54,253 - BERTopic - Dimensionality - Completed ✓
2025-11-06 15:36:54,255 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-06 15:36:54,298 - BERTopic - Cluster - Completed ✓
2025-11-06 15:36:54,299 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-11-06 15:36:54,864 - BERTopic - Representation - Completed ✓
2025-11-06 15:36:54,866 - BERTopic - Topic reduction - Reducing number of topics
2025-11-06 15:36:54,883 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-06 15:36:55,550 - BERTopic - Representation - Completed ✓
2025-11-06 15:36:55,554 - BERTopic - Topic reduction - Reduced number of topics from 37 to 28



✅ 한국 G 결과:
   토픽: 27개
   Outliers: 0개 (0.0%)

📊 상위 토픽들:
  Topic  0: 203 docs - 0_esg_경영_기업_사회
  Topic  1: 146 docs - 1_금융_대표_회장_사장
  Topic  2:  44 docs - 2_업무협약_협약_체결_협력
  Topic  3:  41 docs - 3_iso_인증_획득_국제
  Topic  4:  36 docs - 4_지원_한국_투자_기업
  Topic  5:  32 docs - 5_상장_한진_더본코리아_금융
  Topic  6:  24 docs - 6_배터리_삼성_금속_cme
  Topic  7:  22 docs - 7_등급_평가_esg_msci


In [27]:
# ============================================================
# 결과 저장
# ============================================================
print("="*70)
print("💾 한국 ESG 분석 결과 저장")
print("="*70)

# 1. 한국 E 저장
kr_e_final = kr_e.copy()
kr_e_final['topic'] = topic_model_kr
kr_e_final['topic_prob'] = topic_model_kr
kr_e_final.to_csv('korea_e_with_topics.csv', index=False)
print(f"✅ korea_e_with_topics.csv ({len(kr_e_final):,}개)")

# 2. 한국 S 저장
kr_s_final = kr_s_filtered.copy()
kr_s_final['topic'] = kr_s_topics
kr_s_final['topic_prob'] = kr_s_probs
kr_s_final.to_csv('korea_s_with_topics.csv', index=False)
print(f"✅ korea_s_with_topics.csv ({len(kr_s_final):,}개)")

# 3. 한국 G 저장
kr_g_final = kr_g_filtered.copy()
kr_g_final['topic'] = kr_g_topics
kr_g_final['topic_prob'] = kr_g_probs
kr_g_final.to_csv('korea_g_with_topics.csv', index=False)
print(f"✅ korea_g_with_topics.csv ({len(kr_g_final):,}개)")

# 4. 토픽 정보 저장
kr_e_info = topic_model_kr.get_topic_info()
kr_s_info = kr_s_model.get_topic_info()
kr_g_info = kr_g_model.get_topic_info()

kr_e_info.to_csv('korea_e_topic_info.csv', index=False)
kr_s_info.to_csv('korea_s_topic_info.csv', index=False)
kr_g_info.to_csv('korea_g_topic_info.csv', index=False)

print(f"\n✅ 토픽 정보도 저장 완료!")

# ============================================================
# 전체 요약
# ============================================================
print(f"\n" + "="*70)
print("📊 한국 ESG 분석 완료!")
print("="*70)

summary_kr = pd.DataFrame({
    'Category': ['E', 'S', 'G', 'Total'],
    'Articles': [
        len(kr_e_final),
        len(kr_s_final),
        len(kr_g_final),
        len(kr_e_final) + len(kr_s_final) + len(kr_g_final)
    ],
    'Topics': [15, 6, 27, 48],
    'Outliers': ['0%', '0%', '0%', '0%']
})

print(summary_kr.to_string(index=False))

print(f"\n🎉 All BERTopic analyses complete!")
print(f"   한국: E(15) + S(6) + G(27) = 48개 토픽")
print(f"   영국: E(11) + S(4) + G(2) = 17개 토픽")

💾 한국 ESG 분석 결과 저장
✅ korea_e_with_topics.csv (1,360개)
✅ korea_s_with_topics.csv (555개)
✅ korea_g_with_topics.csv (1,084개)

✅ 토픽 정보도 저장 완료!

📊 한국 ESG 분석 완료!
Category  Articles  Topics Outliers
       E      1360      15       0%
       S       555       6       0%
       G      1084      27       0%
   Total      2999      48       0%

🎉 All BERTopic analyses complete!
   한국: E(15) + S(6) + G(27) = 48개 토픽
   영국: E(11) + S(4) + G(2) = 17개 토픽
